In [ ]:
!pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score

In [ ]:
ROOT = Path("dataset")

PATH_NO_LABEL = ROOT / "spotify_songs.csv"
PATH_INT_LABEL = ROOT / "spotify_songs_with_genre_int.csv"

URL = "https://raw.githubusercontent.com/hyeonx3/ML_Project/main/dataset/spotify_songs_with_genre_int.csv"

try:
    df = pd.read_csv(PATH_INT_LABEL) 
except:
    df = pd.read_csv(URL)
df = df.dropna()

print("=== 클래스 분포 ===")
print(df["genre_int"].value_counts())
print("\n=== 데이터셋 크기 ===")
print("총 샘플 수:", len(df))

=== 클래스 분포 ===
genre_int
5    6043
1    5743
0    5507
3    5431
4    5153
2    4951
Name: count, dtype: int64

=== 데이터셋 크기 ===
총 샘플 수: 32828


In [ ]:
# 특징/레이블 컬럼 정의
num_cols = ["loudness", "danceability", "energy",
            "speechiness", "acousticness", "instrumentalness", "liveness",
            "valence", "tempo", "duration_ms"]
cat_cols = ["key", "mode"]

# 전처리 파이프라인 구성
preprocessor = ColumnTransformer([
    ("num", StandardScaler(), num_cols),
    ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
])

# 데이터 분할
X = df[num_cols + cat_cols]
y = df["genre_int"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.00303, stratify=y, random_state=42
)

# 전처리 적용
X_train_prep = preprocessor.fit_transform(X_train)
X_test_prep = preprocessor.transform(X_test)

print("테스트용 데이터셋 크기:",X_test_prep.shape[0])

테스트용 데이터셋 크기: 100


In [ ]:
# 20개 항목의 라벨 랜덤으로 바꾸기
import numpy as np

np.random.seed(42)

# Get the indices of y_test
indices_to_consider = y_test.index.to_numpy()

# Randomly select 20 unique indices to change
num_to_change = 20
if len(indices_to_consider) < num_to_change:
    num_to_change = len(indices_to_consider)
    print(f"Warning: y_test has only {len(indices_to_consider)} items. Changing all of them.")

selected_indices = np.random.choice(indices_to_consider, size=num_to_change, replace=False)

# Available labels (0 to 5)
all_labels = np.array([0, 1, 2, 3, 4, 5])

# Iterate through the selected indices and change their labels
print("--- y_test에 적용된 변경 사항 ---\n")
for idx in selected_indices:
    original_label = y_test.loc[idx]

    # 현재 레이블과 다른 레이블 목록 생성
    possible_new_labels = all_labels[all_labels != original_label]

    # 가능한 새 레이블 중에서 무작위로 하나 선택
    new_label = np.random.choice(possible_new_labels)

    y_test.loc[idx] = new_label
    print(f"Index: {idx}, Original: {original_label}, New: {new_label}")

print(f"\ny_test에서 총 변경된 레이블 수: {num_to_change}")

--- y_test에 적용된 변경 사항 ---

Index: 29466, Original: 5, New: 2
Index: 28449, Original: 5, New: 0
Index: 10185, Original: 1, New: 3
Index: 32747, Original: 5, New: 2
Index: 11236, Original: 1, New: 0
Index: 130, Original: 0, New: 3
Index: 20330, Original: 4, New: 5
Index: 28281, Original: 5, New: 1
Index: 7383, Original: 1, New: 2
Index: 22348, Original: 3, New: 0
Index: 466, Original: 0, New: 4
Index: 13747, Original: 2, New: 0
Index: 9989, Original: 1, New: 4
Index: 17743, Original: 4, New: 1
Index: 26175, Original: 3, New: 0
Index: 4426, Original: 0, New: 5
Index: 29343, Original: 5, New: 2
Index: 31767, Original: 5, New: 3
Index: 9768, Original: 1, New: 3
Index: 13760, Original: 2, New: 3

y_test에서 총 변경된 레이블 수: 20


In [ ]:
# === LightGBM Classifier ===
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=-1,
    verbosity=-1,
    n_jobs=-1,
    random_state=42
)

lgb.fit(X_train_prep, y_train)

LGBMClassifier(learning_rate=0.05, n_estimators=400, n_jobs=-1, random_state=42,
               verbosity=-1)

In [ ]:
# 예측 확률 계산
y_pred_proba_lgb = lgb.predict_proba(X_test_prep)
TRESHOLD = 0.67

# y_test와 y_pred_lgb를 DataFrame으로 변환하여 인덱스를 맞춥니다.
results_df = pd.DataFrame({
    'actual_label': y_test.values,
    'predicted_label': y_pred_lgb
}, index=y_test.index)

# 예측이 틀린 항목들 필터링
incorrect_predictions = results_df[results_df['actual_label'] != results_df['predicted_label']]

# 기준치를 넘어서지만 라벨과 일치하지 않는 항목들을 저장할 리스트
high_confidence_incorrect = []

print(f"기준치(TRESHOLD = {TRESHOLD})는 넘었지만, 예측이 틀린 항목들:\n")

# 각 틀린 예측에 대해 예측 확률 확인
cnt = 0
for index, row in incorrect_predictions.iterrows():
    actual_idx_in_X_test = y_test.index.get_loc(index) # X_test_prep에서의 인덱스

    predicted_class = row['predicted_label']
    probability_of_predicted_class = y_pred_proba_lgb[actual_idx_in_X_test, predicted_class]

    if probability_of_predicted_class >= TRESHOLD:
        high_confidence_incorrect.append({
            'index': index,
            'actual_label': row['actual_label'],
            'predicted_label': row['predicted_label'],
            'probability': probability_of_predicted_class
        })
        cnt += 1
        print(f"  Index: {index}, 실제 라벨: {row['actual_label']}, 예측 라벨: {row['predicted_label']}, 예측 확률: {probability_of_predicted_class:.4f}")

print()
if not high_confidence_incorrect:
    print("  해당하는 항목이 없습니다.")
else:
    print(f"총 {cnt}개의 항목이 잘못된 라벨이 붙어있을 가능성이 있습니다")


기준치(TRESHOLD = 0.67)는 넘었지만, 예측이 틀린 항목들:

  Index: 22348, 실제 라벨: 0, 예측 라벨: 5, 예측 확률: 0.7757
  Index: 7383, 실제 라벨: 2, 예측 라벨: 1, 예측 확률: 0.7307
  Index: 21270, 실제 라벨: 4, 예측 라벨: 1, 예측 확률: 0.6720
  Index: 20330, 실제 라벨: 5, 예측 라벨: 4, 예측 확률: 0.6762
  Index: 22296, 실제 라벨: 3, 예측 라벨: 1, 예측 확률: 0.8183
  Index: 5226, 실제 라벨: 0, 예측 라벨: 3, 예측 확률: 0.7754
  Index: 13760, 실제 라벨: 3, 예측 라벨: 2, 예측 확률: 0.8247
  Index: 17743, 실제 라벨: 1, 예측 라벨: 4, 예측 확률: 0.7910
  Index: 1685, 실제 라벨: 0, 예측 라벨: 2, 예측 확률: 0.7931
  Index: 11236, 실제 라벨: 0, 예측 라벨: 1, 예측 확률: 0.8213
  Index: 32747, 실제 라벨: 2, 예측 라벨: 5, 예측 확률: 0.8890
  Index: 28449, 실제 라벨: 0, 예측 라벨: 5, 예측 확률: 0.9169
  Index: 31767, 실제 라벨: 3, 예측 라벨: 5, 예측 확률: 0.9320
  Index: 29466, 실제 라벨: 2, 예측 라벨: 5, 예측 확률: 0.6732

총 14개의 항목이 잘못된 라벨이 붙어있을 가능성이 있습니다


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
print("\n--- 잘못된 라벨 가능성 있는 항목과 의도적으로 변경한 항목 비교 ---\n")

# 'high_confidence_incorrect' 리스트에서 인덱스만 추출
identified_incorrect_indices = {item['index'] for item in high_confidence_incorrect}

# 이전에 라벨을 변경한 인덱스 (selected_indices는 numpy array)
previously_changed_indices = set(selected_indices)

# 두 집합의 교집합 (모델이 '높은 확신으로 틀렸다'고 예측한 것 중 우리가 실제로 바꾼 것)
overlap_indices = identified_incorrect_indices.intersection(previously_changed_indices)

# 모델이 '높은 확신으로 틀렸다'고 예측했지만 우리가 바꾸지 않은 것
model_identified_but_not_changed = identified_incorrect_indices - previously_changed_indices

# 우리가 바꿨지만 모델이 '높은 확신으로 틀렸다'고 식별하지 못한 것
changed_but_not_identified_by_model = previously_changed_indices - identified_incorrect_indices

print(f"이전에 라벨을 변경한 총 항목 수: {len(previously_changed_indices)}")
print(f"모델이 높은 확신으로 잘못 예측했다고 식별한 총 항목 수: {len(identified_incorrect_indices)}")

print(f"\n1. 모델이 잘못 예측했다고 식별했으며, 실제로 라벨이 변경된 항목 (교집합): {len(overlap_indices)}개")
if overlap_indices:
    print(f"   인덱스: {sorted(list(overlap_indices))}")

print(f"\n2. 모델이 잘못 예측했다고 식별했지만, 실제로는 라벨을 변경하지 않은 항목: {len(model_identified_but_not_changed)}개")
if model_identified_but_not_changed:
    print(f"   인덱스: {sorted(list(model_identified_but_not_changed))}")

print(f"\n3. 라벨은 변경했지만, 모델이 잘못 예측했다고 식별하지 못한 항목: {len(changed_but_not_identified_by_model)}개")
if changed_but_not_identified_by_model:
    print(f"   인덱스: {sorted(list(changed_but_not_identified_by_model))}")


--- 잘못된 라벨 가능성 있는 항목과 의도적으로 변경한 항목 비교 ---

이전에 라벨을 변경한 총 항목 수: 20
모델이 높은 확신으로 잘못 예측했다고 식별한 총 항목 수: 14

1. 모델이 잘못 예측했다고 식별했으며, 실제로 라벨이 변경된 항목 (교집합): 10개
   인덱스: [7383, 11236, 13760, 17743, 20330, 22348, 28449, 29466, 31767, 32747]

2. 모델이 잘못 예측했다고 식별했지만, 실제로는 라벨을 변경하지 않은 항목: 4개
   인덱스: [1685, 5226, 21270, 22296]

3. 라벨은 변경했지만, 모델이 잘못 예측했다고 식별하지 못한 항목: 10개
   인덱스: [np.int64(130), np.int64(466), np.int64(4426), np.int64(9768), np.int64(9989), np.int64(10185), np.int64(13747), np.int64(26175), np.int64(28281), np.int64(29343)]
